출처 : [Pytorch Tensors](https://pytorch.org/tutorials/beginner/basics/tensorqs_tutorial.html)  
  
Tensor는 배열과 행렬과 비슷한 특별한 자료구조이다. PyTorch에서 모델의 input, output과 parameter를 인코딩하는데 tensor를 사용한다.  
  
Tensor는 Numpy의 ndarrays와 비슷하지만, tensor는 GPU 가속이 가능하다. 또 tensor는 자동 미분(automatic differentiation)에 최적화되어있다.

In [1]:
import torch
import numpy as np

## Initializing a Tensor
Tensor는 다양한 방법으로 초기화할 수 있다.
### Directly from data
Data로 부터 바로 tensor를 만들 수 있다. 여기서 데이터 타입은 자동으로 추론된다.

In [2]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

### From a NumPy array
NumPy array로 부터 tensor를 만들 수 있다. (반대 방향도 가능)

In [3]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

### From another tensor:
다른 tensor의 속성(shape, datatype)으로 부터 새 tensor를 만들 수 있다.

In [4]:
x_ones = torch.ones_like(x_data) # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float) # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Random Tensor: 
 tensor([[0.3985, 0.1358],
        [0.7151, 0.1586]]) 



### With random or constant values:
`shape`를 지정해서 tensor를 만들 수 있다.

In [5]:
shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

Random Tensor: 
 tensor([[0.9729, 0.5650, 0.1026],
        [0.8679, 0.1286, 0.8256]]) 

Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])


## Attributes of a Tensor
Tensor 속성으로 shape, datatype, 그리고 어느 device에 저장되어있는 지가 있다.

In [6]:
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


## Operations on Tensors
Tensor의 연산으로 산술(arithmetic), 선형대수, 행렬 조작(transposing, indexing, slicing), 샘플링 등 100개가 넘는 연산이 있다.  
더 자세한 건 [여기](https://pytorch.org/docs/stable/torch.html) 참고  
  
각 연산은 GPU에서 실행할 수 있다. 기본값으로 tensor는 CPU에서 생성된다. GPU를 사용하려면 명시적으로 `.to` 메소드를 사용해 tensor를 GPU로 이동시켜야 한다(GPU를 사용할 수 있는 지 체크한 이후에). 거대한 tensor를 devices 간에 복사하는 건 큰 비용이 드는 것을 명심해야한다.

In [7]:
# We move our tensor to the GPU if available
if torch.cuda.is_available():
    tensor = tensor.to("cuda")

몇몇 연산을 살펴보는데, NumPy API에 친숙하다면 Tensor API는 사용하기 쉬울 것이다.
### Standard numpy-like indexing and slicing:

In [8]:
tensor = torch.ones(4, 4)
print(f"First row: {tensor[0]}")
print(f"First column: {tensor[:, 0]}")
print(f"Last column: {tensor[..., -1]}")
tensor[:,1] = 0
print(tensor)

First row: tensor([1., 1., 1., 1.])
First column: tensor([1., 1., 1., 1.])
Last column: tensor([1., 1., 1., 1.])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


### Joining tensors
`torch.cat`을 사용하면 tensor들의 시퀀스를 합칠(concatenate) 수 있다. [torch.stack](https://pytorch.org/docs/stable/generated/torch.stack.html)은 `torch.cat`과 다른 합치는 연산이다.

In [9]:
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)

tensor([[1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.]])


### Arithmetic operations

In [10]:
# This computes the matrix multiplication between two tensors.
# y1, y2, y3 will have the same value
# ``tensor.T`` returns the transpose of a tensor
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)

y3 = torch.rand_like(y1)
torch.matmul(tensor, tensor.T, out=y3)


# This computes the element-wise product. z1, z2, z3 will have the same value
z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])

### Single-element tensors
만약에 한 개의 원소를 가진 tensor가 있다면, 예를 들어 모든 값들을 집계해서 하나의 값을 가진 tensor를 만든다면, `item()`을 사용해서 Python 값(variable)로 변환할 수 있다.

In [11]:
agg = tensor.sum()
agg_item = agg.item()
print(agg_item, type(agg_item))

12.0 <class 'float'>


### In-place operations
연산의 결과로 피연산자(operand)에 저장되는 것을 in-place라 한다. inplace 연산은 `_` suffix로 나타낸다. 예: `x.copey_(y)`, `x.t_()` 는 x를 바꾼다. In-place 연산은 메모리를 아끼지만, 미분을 계산하는데 문제가 생길 수 있다. 그래서 사용이 권장되지 않는다.

In [12]:
print(f"{tensor} \n")
tensor.add_(5)
print(tensor)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]]) 

tensor([[6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.]])


## Bridge with Numpy
CPU에 있는 tensor와 Numpy array는 메모리 위치를 공유하고 있어서, 하나를 바꾼다면 다른 하나도 바뀐다.

In [13]:
t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n: {n}")

t: tensor([1., 1., 1., 1., 1.])
n: [1. 1. 1. 1. 1.]


In [14]:
t.add_(1)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.])
n: [2. 2. 2. 2. 2.]


### Numpy array to Tensor

In [15]:
n = np.ones(5)
t = torch.from_numpy(n)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([1., 1., 1., 1., 1.], dtype=torch.float64)
n: [1. 1. 1. 1. 1.]


In [16]:
np.add(n, 1, out=n)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.], dtype=torch.float64)
n: [2. 2. 2. 2. 2.]
